In [1]:
import pymongo
import pandas as pd
import numpy as np

In [2]:

myclient = pymongo.MongoClient('mongodb://localhost:27017')
mydb = myclient["mydatabase"]
words = mydb.p_and_p


In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def get_node_activations(word,breadth,graph=words ,context=[]):
    df  = pd.DataFrame(graph.find({'word':word}))
    if len(df) > 0 :
        breadth = min(len(df),breadth)
        actication_score =  softmax(df['frequency'].values)
        #get index of max n 
        active_index = np.argpartition(actication_score,-breadth)[-breadth:]
        connections = list(df['connection'][active_index])
        return actication_score, connections
    else :
        print(word ,'  : word not in graph')
        return [], []
    
    
def get_branch_activations(word,breadth,depth) :
    response =  [[[word] ,0 , 0]]
    output   = []
    while len(response) > 0 : 
        if response[0][2] < depth:
            activation_score ,connections = get_node_activations(response[0][0][-1],breadth)
            p_node = response[0].copy()
            response.pop(0)
            for index, connection in enumerate(connections):
                branch = p_node[0] + [connection]
                score  = p_node[1] + activation_score[index]
                level  = p_node[2] + 1
                response.append([branch , score ,level])
        else :
            output.append(response[0][:-1])
            response.pop(0)
    return output

In [117]:
depth =  1
breadth = 4
word = 'now'
get_branch_activations(word,breadth,depth)

[[['now', 'be'], 0.9852457424535535],
 [['now', 'to'], 1.6455279595322684e-05],
 [['now', 'and'], 4.4730087506178566e-05],
 [['now', '.'], 0.00012158898405342818]]

In [118]:
depth =  2
breadth = 4
word = 'now'
get_branch_activations(word,breadth,depth)

[[['now', 'be', 'the'], 1.7114113922627048],
 [['now', 'be', 'a'], 0.9852457424535536],
 [['now', 'be', 'so'], 0.9852457424809672],
 [['now', 'be', 'in'], 1.2523871559032413],
 [['now', 'to', 'his'], 1.6455279595322684e-05],
 [['now', 'to', 'her'], 1.6455279595322684e-05],
 [['now', 'to', 'the'], 1.6455279595322684e-05],
 [['now', 'to', 'be'], 1.6455279595322684e-05],
 [['now', 'and', 'her'], 4.4730087506178566e-05],
 [['now', 'and', 'the'], 4.473008750617857e-05],
 [['now', 'and', 'I'], 4.4730087506178566e-05],
 [['now', 'and', 'she'], 4.4730087506178566e-05],
 [['now', '.', 'Darcy'], 0.00012158898405342818],
 [['now', '.', 'But'], 0.00012158898405342818],
 [['now', '.', 'She'], 0.00012158898405342818],
 [['now', '.', 'I'], 0.00012158898405342818]]

In [120]:
depth =  4
breadth = 2
word = 'now'
get_branch_activations(word,breadth,depth)

[[['now', 'and', 'I', 'have', 'no'], 0.9852457424535535],
 [['now', 'and', 'I', 'have', 'been'], 0.9852457424535535],
 [['now', 'and', 'I', 'am', 'not'], 1.9852457424535535],
 [['now', 'and', 'I', 'am', 'sure'], 2.9852457424532988],
 [['now', 'and', 'the', 'room', 'and'], 0.985245742453647],
 [['now', 'and', 'the', 'room', '.'], 0.985245742453647],
 [['now', 'and', 'the', 'same', '.'], 1.9852385795015017],
 [['now', 'and', 'the', 'same', 'time'], 0.985245854987922],
 [['now', '.', 'She', 'had', 'not'], 1.0000163427305453],
 [['now', '.', 'She', 'had', 'been'], 1.0000163427305453],
 [['now', '.', 'She', 'was', 'a'], 1.6455279595322762e-05],
 [['now', '.', 'She', 'was', 'not'], 1.6455279595322762e-05],
 [['now', '.', 'I', 'have', 'no'], 1.6455279595322684e-05],
 [['now', '.', 'I', 'have', 'been'], 1.6455279595322684e-05],
 [['now', '.', 'I', 'am', 'not'], 1.0000164552795954],
 [['now', '.', 'I', 'am', 'sure'], 2.000016455279341]]

In [5]:
depth =  5
breadth = 1
word = 'the'
get_branch_activations(word,breadth,depth)

[[['the', 'same', 'time', '.', 'I', 'am'], 0.9999949901330827]]